In [ ]:
%autosave 0
import numpy as np 
import pandas as pd 
from pandas import read_csv, DataFrame
from math import sqrt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras import Sequential
from keras.layers import Dense, Dropout, Input, Activation
from keras import metrics
from keras import optimizers
from keras import regularizers

from keras.wrappers.scikit_learn import KerasRegressor

import seaborn as sns
import matplotlib.pyplot as plt

# ignore Warning
import warnings
warnings.filterwarnings("ignore") 



In [ ]:
#### Mettre la variable sur True si recherche des paramétres opti, mais l'opération dure environ 45 minutes
lookForOptimiz = True

In [ ]:
df_train = read_csv('new_Basetrain.csv')
df_Kagg = read_csv('new_kaggtest.csv')

submission = pd.DataFrame()
submission["Id"]= df_Kagg["Id"]

# Preparation du split

In [ ]:
X_train = df_train

V_train = df_train['target'].values
F_train = df_train.drop(['target','SalePriceLog'], axis=1)

F_train

In [ ]:
V_train

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(F_train, V_train, test_size=0.3, random_state=seed)

scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val = scale.transform(X_val)
df_Kagg = scale.transform(df_Kagg)

y_scale = StandardScaler()
y_scale.fit(y_train.reshape(-1, 1))
y_train = y_scale.transform(y_train.reshape(-1, 1))
y_val_resc = y_scale.transform(y_val.reshape(-1, 1))

df_Kagg

# Recherche des paramétres opti du réseau

In [ ]:
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(200, input_dim=X_train.shape[1], kernel_regularizer=regularizers.l1(0.01)))
    model.add(Activation('relu')) 
#     model.add(Dense(100, kernel_initializer='normal'))
#     model.add(Activation('relu'))
    model.add(Dense(1, kernel_initializer='normal',kernel_regularizer=regularizers.l1(0.01)))
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [ ]:
def BestParams():
    estimators = []
    #estimators.append(('pf', PolynomialFeatures(interaction_only=True,include_bias=False)))
    estimators.append(('ss', StandardScaler()))
    estimators.append(('nn', KerasRegressor(build_fn=create_model, epochs=10, batch_size=5, verbose=0)))
    nn_pipe = Pipeline(estimators)

    nn_param_grid = {'nn__epochs': [1000,1500,2000],'nn__batch_size':[5,100,200]}

    gs_nn = GridSearchCV(nn_pipe, nn_param_grid,verbose=0, cv=3)
    print('Ok')
    gs_nn.fit(X_train, y_train)
    return gs_nn.best_params_

In [ ]:
X_train.shape, y_train.shape

In [ ]:
if lookForOptimiz : 
    best = BestParams()
    print(best)

# Creation du modèle

In [ ]:
model = create_model()

In [ ]:
m_fti = model.fit(X_train, y_train, epochs=2000, batch_size=200)

In [ ]:
DataFrame(m_fti.history).plot()
plt.grid = True
plt.show

In [ ]:
X_val.shape

In [ ]:
pred = model.predict(X_val)
pred = y_scale.inverse_transform(pred)

In [ ]:
RMSE = np.sqrt(MSE(pred,y_val))
print(RMSE.round(4))

In [ ]:
pred_kagg = model.predict(df_Kagg)

In [ ]:
pred_kagg.shape

In [ ]:
pred_kagg = y_scale.inverse_transform(pred_kagg)

In [ ]:
pred_kagg


In [ ]:
submission["SalePrice"]= pred_kagg
submission.to_csv("new_kagg10.csv", index= False)

In [ ]:
from IPython.display import Image
Image('png/Neuro.png')